In [1]:
import numpy as np
from skyfield.api import load, wgs84, Distance
from skyfield.toposlib import ITRSPosition
from skyfield.framelib import itrs
from coverage_v2 import *
from scipy.spatial.transform import Rotation as R

import geopandas as gpd
from shapely.geometry import Polygon

In [2]:
tles = gen_sats(sat_nos=[49260])
sat = tles[0][0]
times = gen_times(start_yr=2021, start_mo=11, start_day=27, days=1, step_min=1)

hfov_x = 15
hfov_y = 10


Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2021-12-04 18:44:33 UTC>]
Propogation time: 
 2021-11-27 00:00:00+00:00 
to 
 2021-11-27 23:59:00+00:00


In [3]:
# https://github.com/skyfielders/python-skyfield/issues/492
# https://github.com/skyfielders/python-skyfield/issues/557

# lvlh issues

In [4]:
inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True)

hfov_x = inst["hfov_deg"]/2
hfov_y = inst["vfov_deg"]/2

cs_dict = {"c1": {"X" : -hfov_x, "Y": hfov_y}, 
    "c2": {"X" : hfov_x, "Y": hfov_y},
    "c3": {"X" : hfov_x, "Y": -hfov_y},
    "c4": {"X" : -hfov_x, "Y": -hfov_y}
}

In [5]:
polys = []

for time in times:

    SSP_lat, SSP_lon, SSP_d, lvlh, vec = get_los2(sat, time)
    xyz_rates = sat.at(time).frame_xyz_and_velocity(itrs)
    xyz_dist = xyz_rates[0]

    cs_lla_dict = {"c1": {"lat" : None, "lon": None}, 
        "c2": {"lat" : None, "lon": None},
        "c3": {"lat" : None, "lon": None},
        "c4": {"lat" : None, "lon": None}
    }

    for c in cs_dict:
        # Generate X and Y rotation vectors
        rot_X_deg = cs_dict[c]["X"]
        rot_X_rad = np.radians(rot_X_deg)
        rot_X_ax = lvlh["X"]

        rot_Y_deg = cs_dict[c]["Y"]
        rot_Y_rad = np.radians(rot_Y_deg)
        rot_Y_ax = lvlh["Y"]

        # Rotate about X
        rot_X_vec = rot_X_rad * rot_X_ax
        rot_X = R.from_rotvec(rot_X_vec)
        los_X = rot_X.apply(vec)

        # Rotate about Y
        rot_Y_vec = rot_Y_rad * rot_Y_ax
        rot_Y = R.from_rotvec(rot_Y_vec)
        los_XY = rot_Y.apply(los_X)

        los_xyz = los_to_earth(xyz_dist.km, los_XY)
        los = Distance(km=los_xyz)
        los_itrs = ITRSPosition(los)
        los_itrs.at(time).frame_xyz(itrs).km

        los_lat, los_lon = wgs84.latlon_of(los_itrs.at(time))
        cs_lla_dict[c]["lat"] = los_lat.degrees
        cs_lla_dict[c]["lon"] = los_lon.degrees

    polys.append(Polygon([(cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"]), 
            (cs_lla_dict["c2"]["lon"], cs_lla_dict["c2"]["lat"]), 
            (cs_lla_dict["c3"]["lon"], cs_lla_dict["c3"]["lat"]),
            (cs_lla_dict["c4"]["lon"], cs_lla_dict["c4"]["lat"]),
            (cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"])]
            # (gdf.loc[n].bound1_lon, gdf.loc[n].bound1_lat)
            ))
        

In [6]:
cs_lla_dict

{'c1': {'lat': 80.30467693453419, 'lon': -94.69051271162316},
 'c2': {'lat': 81.85487120839703, 'lon': -90.31603328503128},
 'c3': {'lat': 82.4227424159076, 'lon': -101.52519994596487},
 'c4': {'lat': 80.7770985389288, 'lon': -104.20530888205397}}

In [7]:
poly_df = gpd.GeoDataFrame(
        data=polys, 
        columns=['geometry'], 
        crs="EPSG:4326"
        )

In [8]:
poly_plot = poly_df.geometry.to_crs("ESRI:54009")
poly_plot = poly_plot[poly_plot.area < poly_plot.area.quantile(.96)]
poly_plot.explore()

In [9]:
poly_df.to_crs("ESRI:54009").to_parquet("v2_test.parquet")

/var/folders/fc/b2n8k5vx0fd5pss09qxrjkd80000gn/T/ipykernel_57762/2935036610.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  poly_df.to_crs("ESRI:54009").to_parquet("v2_test.parquet")
